In [1]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC, AutoTokenizer
import librosa
import torch
import numpy as np
import re
import json
from tqdm.auto import tqdm

from src.model import predict, align

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = "nguyenvulebinh/wav2vec2-base-vietnamese-250h"
processor = Wav2Vec2Processor.from_pretrained(model_name)
model = Wav2Vec2ForCTC.from_pretrained(model_name)
model.to(device)
print()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/opt/anaconda3/envs/ml/lib/python3.9/site-packages/transformers/configuration_utils.py:369: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [3]:
TRAIN_DIR = f'data/train/'
TEST_DIR = f'data/public_test'
VERSION = '250h_pretrained'
OUT_DIR = f'submissions/{VERSION}'
SAMPLING_RATE = 16000

In [4]:
os.makedirs(OUT_DIR, exist_ok=True)

In [5]:
for fname in tqdm(os.listdir(f'{TEST_DIR}/songs/')):
    name = fname.split('.')[0]
    wav, _ = librosa.load(f'{TEST_DIR}/songs/{name}.wav',
                            sr = SAMPLING_RATE
    )
    lyric = np.loadtxt(f'{TEST_DIR}/lyrics/{name}.txt', dtype=str).tolist()

    # lyric = []

    # for ele in label:
    #     lyric += ([w['d'] for w in ele['l']])
        
    lookup = predict(processor, model, wav)
    struct = align(lyric, lookup)

    with open(f'{OUT_DIR}/{name}.json', 'w', encoding='utf8') as f:
        json.dump(struct, f, ensure_ascii=False)
    # break

  0%|          | 0/264 [00:00<?, ?it/s]